IMPORT ALL THE REQUIRED LIBRARIES

In [1]:
import os
import time
import torch
import numpy as np
import pandas as pd
import torch.nn as nn
from scipy import linalg
import torch.optim as optim
import matplotlib.pyplot as plt
import torch.nn.functional as F
from IPython.display import HTML
import torchvision.utils as utils
from torch.utils.data import Subset
import torchvision.models as models
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
import matplotlib.animation as animation
import torchvision.transforms as transforms

DEFINE THE PARAMETERS

In [2]:
torch.manual_seed(1)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

LEARNING_RATE = 2e-4
BATCH_SIZE = 128
IMAGE_SIZE = 64
CHANNELS_IMG = 3  
NOISE_DIM = 100
NUM_EPOCHS = 30
FEATURES_DISC = 64 
FEATURES_GEN = 64 
criticItr = 5
lambda_GP = 10

LOAD THE DATASET

In [3]:
dataset = datasets.CIFAR10(root="./CIFAR_dataset/CIFAR10data", download=False,
                           transform=transforms.Compose([
                               transforms.Resize(64),
                               transforms.ToTensor(),
                               transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                           ]))

dataloader = torch.utils.data.DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)

DEFINE THE DISCRIMINATOR

In [4]:
class Discriminator(nn.Module):
    def __init__(self, channels_img, features_d):
        super(Discriminator, self).__init__()
        self.disc = nn.Sequential(
            nn.Conv2d(
                channels_img, features_d, kernel_size=4, stride=2, padding=1
            ),
            nn.LeakyReLU(0.2),
            self.Dnet(features_d, features_d * 2, 4, 2, 1),     
            self.Dnet(features_d * 2, features_d * 4, 4, 2, 1), 
            self.Dnet(features_d * 4, features_d * 8, 4, 2, 1),  
            nn.Conv2d(features_d * 8, 1, kernel_size=4, stride=2, padding=0),
        )
        
    def Dnet(self, in_channels, out_channels, kernel_size, stride, padding):
        return nn.Sequential(
            nn.Conv2d(
                in_channels,
                out_channels,
                kernel_size,
                stride,
                padding,
                bias=False,
            ),
            nn.InstanceNorm2d(out_channels, affine =True),
            nn.LeakyReLU(0.2),
        )

    def forward(self, x):
        return self.disc(x)

DEFINE THE GENERATOR

In [5]:
class Generator(nn.Module):
    def __init__(self, channels_noise, channels_img, features_g):
        super(Generator, self).__init__()
        self.net = nn.Sequential(
            self.Gnet(channels_noise, features_g*8, 4, 1, 0),        
            self.Gnet(features_g*8, features_g*4, 4, 2, 1),
            self.Gnet(features_g*4, features_g*2, 4, 2, 1), 
            self.Gnet(features_g*2, features_g, 4, 2, 1),
            nn.ConvTranspose2d(features_g, channels_img, kernel_size=4, stride=2, padding=1),
            nn.Tanh() 
        
        )
        
    def Gnet(self, in_channels, out_channels, kernel_size, stride, padding):
        return nn.Sequential(
            nn.ConvTranspose2d(
                in_channels,
                out_channels,
                kernel_size,
                stride,
                padding,
                bias=False,
            ),
            nn.BatchNorm2d(out_channels,momentum=0.9),
            nn.ReLU(),
        )

    def forward(self, x):
        return self.net(x)

In [6]:
def initialize_weights(model):
    for m in model.modules():
        if isinstance(m, (nn.Conv2d, nn.ConvTranspose2d, nn.BatchNorm2d)):
            nn.init.normal_(m.weight.data, 0.0, 0.02)

In [7]:
def GP (critic, real, fake):
    BATCH_SIZE, C, H, W = real.shape
    alpha = torch.rand((BATCH_SIZE, 1, 1, 1)).repeat(1, C, H, W).to(device)
    interpolated_images = real * alpha + fake * (1 - alpha)
    mixed_scores = critic(interpolated_images)
    gradient = torch.autograd.grad(
        inputs=interpolated_images,
        outputs=mixed_scores,
        grad_outputs=torch.ones_like(mixed_scores),
        create_graph=True,
        retain_graph=True,
    )[0]
    gradient = gradient.view(gradient.shape[0], -1)
    gradient_norm = gradient.norm(2, dim=1)
    gradient_penalty = torch.mean((gradient_norm - 1) ** 2)
    return gradient_penalty

In [8]:
def save_model(model):
    if model == 'gen':
        torch.save(model, "Models/CIFAR_wganGP_GENERATOR.pth.tar")
    elif model == 'disc':
        torch.save(model, "Models/CIFAR_wganGP_DIS.pth.tar")
        
def load_model(gen,disc):
    gen = torch.load("Models/CIFAR_wganGP_GENERATOR.pth.tar")
    disc = torch.load("Models/CIFAR_wganGP_DIS.pth.tar")

In [9]:
gen = Generator(NOISE_DIM, CHANNELS_IMG, FEATURES_GEN).to(device)
disc = Discriminator(CHANNELS_IMG, FEATURES_DISC).to(device)
initialize_weights(gen)
initialize_weights(disc)
optimGenerator = optim.RMSprop(gen.parameters(), lr=LEARNING_RATE)
optimDiscriminator = optim.RMSprop(disc.parameters(), lr=LEARNING_RATE)
fixed_noise = torch.randn(BATCH_SIZE, NOISE_DIM, 1, 1).to(device)
step = 0

In [10]:
gen.train() , disc.train()

(Generator(
   (net): Sequential(
     (0): Sequential(
       (0): ConvTranspose2d(100, 512, kernel_size=(4, 4), stride=(1, 1), bias=False)
       (1): BatchNorm2d(512, eps=1e-05, momentum=0.9, affine=True, track_running_stats=True)
       (2): ReLU()
     )
     (1): Sequential(
       (0): ConvTranspose2d(512, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
       (1): BatchNorm2d(256, eps=1e-05, momentum=0.9, affine=True, track_running_stats=True)
       (2): ReLU()
     )
     (2): Sequential(
       (0): ConvTranspose2d(256, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
       (1): BatchNorm2d(128, eps=1e-05, momentum=0.9, affine=True, track_running_stats=True)
       (2): ReLU()
     )
     (3): Sequential(
       (0): ConvTranspose2d(128, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
       (1): BatchNorm2d(64, eps=1e-05, momentum=0.9, affine=True, track_running_stats=True)
       (2): ReLU()
     )
     (4): ConvTr

In [11]:
class InceptionV3(nn.Module):
    
    DEFAULT_BLOCK_INDEX = 3
    BLOCK_INDEX_BY_DIM = {
        64: 0,   
        192: 1, 
        768: 2, 
        2048: 3 
    }

    def __init__(self,
                 output_blocks=[DEFAULT_BLOCK_INDEX],
                 resize_input=True,
                 normalize_input=True,
                 requires_grad=False):
        
        super(InceptionV3, self).__init__()

        self.resize_input = resize_input
        self.normalize_input = normalize_input
        self.output_blocks = sorted(output_blocks)
        self.last_needed_block = max(output_blocks)
        
        assert self.last_needed_block <= 3, \
            'Last possible output block index is 3'

        self.blocks = nn.ModuleList()
        inception = models.inception_v3(pretrained=True)

        block0 = [
            inception.Conv2d_1a_3x3,
            inception.Conv2d_2a_3x3,
            inception.Conv2d_2b_3x3,
            nn.MaxPool2d(kernel_size=3, stride=2)
        ]
        self.blocks.append(nn.Sequential(*block0))

        if self.last_needed_block >= 1:
            block1 = [
                inception.Conv2d_3b_1x1,
                inception.Conv2d_4a_3x3,
                nn.MaxPool2d(kernel_size=3, stride=2)
            ]
            self.blocks.append(nn.Sequential(*block1))
            
        if self.last_needed_block >= 2:
            block2 = [
                inception.Mixed_5b,
                inception.Mixed_5c,
                inception.Mixed_5d,
                inception.Mixed_6a,
                inception.Mixed_6b,
                inception.Mixed_6c,
                inception.Mixed_6d,
                inception.Mixed_6e,
            ]
            self.blocks.append(nn.Sequential(*block2))

        if self.last_needed_block >= 3:
            block3 = [
                inception.Mixed_7a,
                inception.Mixed_7b,
                inception.Mixed_7c,
                nn.AdaptiveAvgPool2d(output_size=(1, 1))
            ]
            self.blocks.append(nn.Sequential(*block3))

        for param in self.parameters():
            param.requires_grad = requires_grad
        
    def forward(self, inp):
        outp = []
        x = inp

        if self.resize_input:
            x = F.interpolate(x,
                              size=(299, 299),
                              mode='bilinear',
                              align_corners=False)

        if self.normalize_input:
            x = 2 * x - 1 

        for idx, block in enumerate(self.blocks):
            x = block(x)
            if idx in self.output_blocks:
                outp.append(x)

            if idx == self.last_needed_block:
                break

        return outp

block_idx = InceptionV3.BLOCK_INDEX_BY_DIM[2048]
model = InceptionV3([block_idx])
model=model.cuda()

def calculate_activation_statistics(images,model,batch_size=128, dims=2048,
                    cuda=False):
    model.eval()
    act=np.empty((len(images), dims))
    
    if cuda:
        batch=images.cuda()
    else:
        batch=images
    pred = model(batch)[0]
    if pred.size(2) != 1 or pred.size(3) != 1:
        pred = adaptive_avg_pool2d(pred, output_size=(1, 1))

    act= pred.cpu().data.numpy().reshape(pred.size(0), -1)
    
    mu = np.mean(act, axis=0)
    sigma = np.cov(act, rowvar=False)
    return mu, sigma

def calculate_frechet_distance(mu1, sigma1, mu2, sigma2, eps=1e-6):
    mu1 = np.atleast_1d(mu1)
    mu2 = np.atleast_1d(mu2)

    sigma1 = np.atleast_2d(sigma1)
    sigma2 = np.atleast_2d(sigma2)

    assert mu1.shape == mu2.shape, \
        'Training and test mean vectors have different lengths'
    assert sigma1.shape == sigma2.shape, \
        'Training and test covariances have different dimensions'

    diff = mu1 - mu2

    
    covmean, _ = linalg.sqrtm(sigma1.dot(sigma2), disp=False)
    if not np.isfinite(covmean).all():
        msg = ('fid calculation produces singular product; '
               'adding %s to diagonal of cov estimates') % eps
        print(msg)
        offset = np.eye(sigma1.shape[0]) * eps
        covmean = linalg.sqrtm((sigma1 + offset).dot(sigma2 + offset))
    
    if np.iscomplexobj(covmean):
        if not np.allclose(np.diagonal(covmean).imag, 0, atol=1e-3):
            m = np.max(np.abs(covmean.imag))
            raise ValueError('Imaginary component {}'.format(m))
        covmean = covmean.real

    tr_covmean = np.trace(covmean)

    return (diff.dot(diff) + np.trace(sigma1) +
            np.trace(sigma2) - 2 * tr_covmean)

def calculate_fretchet(images_real,images_fake,model):
    mu_1,std_1=calculate_activation_statistics(images_real,model,cuda=True)
    mu_2,std_2=calculate_activation_statistics(images_fake,model,cuda=True)
    
    """get fretched distance"""
    fid_value = calculate_frechet_distance(mu_1, std_1, mu_2, std_2)
    return fid_value

/home/mnarkhe/.local/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/mnarkhe/.local/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=Inception_V3_Weights.IMAGENET1K_V1`. You can also use `weights=Inception_V3_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
GenLoss = []
DiscLoss = []
DiscLossP = []
img_list = []
iters = 0
FID_list = []

print("===== WGAN-GP Training Initialization =====")

print("\n[STATUS] Training Started")
time_start = time.time()

for epoch in range(NUM_EPOCHS):
    epoch = epoch+1
    epoch_gen_loss = 0.0
    epoch_disc_loss = 0.0
    
    print(f"\n[EPOCH {epoch}/{NUM_EPOCHS}] Beginning training")
    
    for batch_idx, data in enumerate(dataloader,0):
        real = data[0].to(device)
        batch = real.shape[0]
        
        for i in range(criticItr):
            noise = torch.randn(batch, NOISE_DIM, 1, 1).to(device)
            fake = gen(noise)
            critic_real = disc(real).reshape(-1)
            critic_fake = disc(fake).reshape(-1)
            gp = GP(disc, real,fake)
            loss_critic = (-(torch.mean(critic_real) - torch.mean(critic_fake))+ (lambda_GP*gp))
            disc.zero_grad()
            loss_critic.backward(retain_graph=True)
            optimDiscriminator.step()
            
            epoch_disc_loss += loss_critic.item()
        
        output = disc(fake).reshape(-1)
        loss_gen = -torch.mean(output) 
        gen.zero_grad()
        loss_gen.backward()
        optimGenerator.step()
        
        epoch_gen_loss += loss_gen.item()
        
        GenLoss.append(loss_gen.detach().cpu())
        DiscLoss.append(loss_critic.detach().cpu())
        
        if (iters % 500 == 0) or ((epoch == NUM_EPOCHS) and (batch_idx == len(dataloader)-1)):
            with torch.no_grad():
                fake = gen(fixed_noise).detach().cpu()
            img_list.append(utils.make_grid(fake, padding=2, normalize=True))
            print(f"[IMAGE SAVE] Saved generated images at iteration {iters}")
            
        iters += 1
    
    fretchet_dist = calculate_fretchet(real,fake,model)
    FID_list.append(fretchet_dist)
    
    print(f"[EPOCH SUMMARY] Epoch {epoch}:")
    print(f"  Average Discriminator Loss: {epoch_disc_loss/criticItr:.4f}")
    print(f"  Average Generator Loss: {epoch_gen_loss:.4f}")
    print(f"  FID Score: {fretchet_dist:.4f}")
    
    labels = torch.arange(0,10,dtype=torch.long,device=device)
    noise = torch.randn(10,100,device=device)  
    images = gen(fixed_noise)
    
    os.makedirs('Results/WGANGP_FAKE', exist_ok=True)
    os.makedirs('Results/WGANGP_REAL', exist_ok=True)
    
    utils.save_image(images.detach(), f'Results/WGANGP_FAKE/WGANGP_epoch_{epoch:03d}.png', normalize=True)
    print(f"[SAVE] Fake images saved for epoch {epoch}")
    
    real = data[0].to(device)
    utils.save_image(real.detach(), f'Results/WGANGP_REAL/WGANGP_epoch_{epoch:03d}.png', normalize=True)
    print(f"[SAVE] Real images saved for epoch {epoch}")
        
time_end = time.time()
total_training_time = time_end - time_start

print("\n===== Training Completed =====")
print(f"Total Training Time: {total_training_time:.2f} seconds")
print(f"Average Time per Epoch: {total_training_time/NUM_EPOCHS:.2f} seconds")

===== WGAN-GP Training Initialization =====

[STATUS] Training Started

[EPOCH 1/30] Beginning training
[IMAGE SAVE] Saved generated images at iteration 0
[EPOCH SUMMARY] Epoch 1:
  Average Discriminator Loss: -12569.5932
  Average Generator Loss: 32620.9774
  FID Score: 228.6168
[SAVE] Fake images saved for epoch 1
[SAVE] Real images saved for epoch 1

[EPOCH 2/30] Beginning training
[IMAGE SAVE] Saved generated images at iteration 500
[EPOCH SUMMARY] Epoch 2:
  Average Discriminator Loss: -5079.7074
  Average Generator Loss: 35449.9105
  FID Score: 218.8400
[SAVE] Fake images saved for epoch 2
[SAVE] Real images saved for epoch 2

[EPOCH 3/30] Beginning training
[IMAGE SAVE] Saved generated images at iteration 1000
[EPOCH SUMMARY] Epoch 3:
  Average Discriminator Loss: -3893.4211
  Average Generator Loss: 35251.5948
  FID Score: 219.9726
[SAVE] Fake images saved for epoch 3
[SAVE] Real images saved for epoch 3

[EPOCH 4/30] Beginning training
[IMAGE SAVE] Saved generated images at it

In [ ]:
np.save('Results/Loss_Data/WGANGP_GLoss', GenLoss) 
np.save('Results/Loss_Data/WGANGP_DLoss', DiscLoss) 
GenLoss = np.load('Results/Loss_Data/WGANGP_GLoss.npy')
DiscLoss = np.load('Results/Loss_Data/WGANGP_DLoss.npy')

In [ ]:
plt.figure(figsize=(10,5))
plt.title("Generator and Discriminator Loss During Training")
plt.plot(GenLoss,label="Generator", color='green')
plt.plot(DiscLoss,label="Discriminator", color='red')
plt.xlabel("Number of iterations")
plt.ylabel("Loss")
plt.legend()
plt.savefig('Results/Loss.pdf',
            format='pdf',
            dpi=100,
            bbox_inches='tight')
plt.show()

In [ ]:
real_batch = next(iter(dataloader))
plt.figure(figsize=(15,15))
plt.subplot(1,2,1)
plt.axis("off")
plt.title("Real Images")
plt.imshow(np.transpose(utils.make_grid(real_batch[0].to(device)[:64], padding=1, normalize=True).cpu(),(1,2,0)))
plt.subplot(1,2,2)
plt.axis("off")
plt.title("Fake Images")
plt.imshow(np.transpose(img_list[-1],(1,2,0)))
plt.savefig('Results/WGANGP_BEST_FAKE_32.pdf',
            format='pdf',
            dpi=100,
            bbox_inches='tight')
plt.show()

In [ ]:
plt.figure(figsize=(10,5))
plt.title("FID Scores for WGANGP")
plt.plot(FID_list,label="WGANGP", color='green')
plt.xlabel("Epochs")
plt.ylabel("FID")
plt.legend()
plt.savefig('Results/FID.pdf',
            format='pdf',
            bbox_inches='tight')
plt.show()

In [ ]:
np.mean(GenLoss),np.mean(DiscLoss),np.min(GenLoss),np.min(DiscLoss),GenLoss[-1],DiscLoss[-1]

In [ ]:
np.min(FID_list), np.max(FID_list), np.mean(FID_list), FID_list[-1]